In [2]:
def get_weather(city: str) -> str:
  return f"It's sunny in {city}"

def summarize_email(email_text: str) -> str:
  return f"Summary: {email_text[:50]}..."

function_registry = {
  "get_weather": get_weather,
  "summarize_email": summarize_email,
}

In [25]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

user_prompt = """
Your job is to assist me by calling the correct function. 
Available functions:
- get_weather(city: str)
- summarize_email(email_text: str)

When appropriate, reply ONLY in JSON like:
{
  "action": "get_weather",
  "args": {"city": "Tokyo"}
}
If no function is needed, reply: {"action": null}.

Now, what's the weather like in kathmandu?
"""
model = ChatOllama(
    model="deepseek-r1:8b",
    temperature=0.8, 
    top_p=0.95,
    top_k=50,
    num_ctx=2048, 
    repeat_penalty=1.0
)

response = model.invoke([HumanMessage(content=user_prompt)])

## parse and run
import json
try:
    content = response.content
    if isinstance(content, list):
        # Convert all elements to strings before joining
        content = "".join(str(x) for x in content)
    response_json = json.loads(content)
    if response_json.get("action"):
        function_name = response_json["action"]
        args = response_json.get("args", {})
        if function_name and function_name in function_registry:
            result = function_registry[function_name](**args)
            print(f"Function call result: {result}")
        elif function_name is None:
            print("No function call needed.")
        else:
            print(f"Function {function_name} not found.")
    else:
        print("No function call needed.")
except json.JSONDecodeError:
    print("Response is not in JSON format:", response.content)



Response is not in JSON format: <think>
Alright, the user is asking about the weather in Kathmandu. Since I don't have real-time data access, I can't provide current weather information. I should let them know that I can't help with that directly. But I can use the available function to get the weather, so I'll call get_weather with Kathmandu as the argument. If they need more info, they can follow up, and I can look it up then.
</think>

```json
{
  "action": "get_weather",
  "args": {
    "city": "Kathmandu"
  }
}
```
